In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from scipy.spatial import Voronoi
from shapely.geometry import Polygon

import sys
sys.path.append('../jamaica-infrastructure')

from scripts.preprocess.preprocess_utils import \
assign_node_weights_by_population_proximity,voronoi_finite_polygons_2d,extract_nodes_within_gdf

In [ ]:
nodes = gpd.read_file('../data/spatial/infrasim-network/nodes.shp')
sinks = nodes[nodes['asset_type']=='sink'].reset_index(drop=True)
sinks = sinks#.head(500)
sinks.head(2)

admin_boundaries = gpd.read_file('../data/spatial/else/admin-boundaries.shp')
admin_boundaries.head(2)

# rename
sinks['parish'] = sinks['parish'].str.replace('Kingston','KSA')
sinks['parish'] = sinks['parish'].str.replace('St. Andrew','KSA')
admin_boundaries['Parish'] = admin_boundaries['Parish'].str.replace('KSA','KSA')
admin_boundaries['Parish'] = admin_boundaries['Parish'].str.replace('St','St.')

In [ ]:
sinks.parish.unique()

In [ ]:
admin_boundaries.Parish.unique()

In [ ]:
node_id_column='id'
pop_id='population'
epsg=3448,
voronoi_path='../data/spatial/electricity_voronoi.shp',

combined_voronoi = []
combined_nodes = []

for unique_parish in sinks.parish.unique():
    
    nodes_dataframe = sinks.loc[sinks.parish.isin([unique_parish])].reset_index(drop=True).copy()
    #population_dataframe = pop_bound.loc[pop_bound.PARISH.isin([unique_parish])].reset_index(drop=True).copy()
    parish_dataframe = admin_boundaries.loc[admin_boundaries.Parish.isin([unique_parish])].reset_index(drop=True).copy()
    
    # load provinces and get geometry of the right population_dataframe
    sindex_parish_dataframe = parish_dataframe.sindex

    # create Voronoi polygons for the nodes
    xy_list = []
    for iter_, values in nodes_dataframe.iterrows():
        xy = list(values.geometry.coords)
        xy_list += [list(xy[0])]

    vor = Voronoi(np.array(xy_list))
    regions, vertices = voronoi_finite_polygons_2d(vor)
    min_x = vor.min_bound[0] - 0.1
    max_x = vor.max_bound[0] + 0.1
    min_y = vor.min_bound[1] - 0.1
    max_y = vor.max_bound[1] + 0.1

    mins = np.tile((min_x, min_y), (vertices.shape[0], 1))
    bounded_vertices = np.max((vertices, mins), axis=0)
    maxs = np.tile((max_x, max_y), (vertices.shape[0], 1))
    bounded_vertices = np.min((bounded_vertices, maxs), axis=0)

    box = Polygon([[min_x, min_y], [min_x, max_y], [max_x, max_y], [max_x, min_y]])

    poly_list = []
    for region in regions:
        polygon = vertices[region]
        # Clipping polygon
        poly = Polygon(polygon)
        poly = poly.intersection(box)
        poly_list.append(poly)

    poly_index = list(np.arange(0, len(poly_list), 1))

    poly_df = pd.DataFrame(list(zip(poly_index, poly_list)),
                                   columns=['gid', 'geometry'])

    gdf_voronoi = gpd.GeoDataFrame(poly_df, geometry = 'geometry') #,crs=f'epsg:{epsg}'
    
    # add nodes
    gdf_voronoi[node_id_column] = gdf_voronoi.progress_apply(    
            lambda row: extract_nodes_within_gdf(row['geometry'],nodes_dataframe,node_id_column),axis=1)
    
    # add parish
    gdf_voronoi['Parish'] = gdf_voronoi[node_id_column].map(nodes_dataframe.set_index('id')['parish'].to_dict())
    
    # rename geom cols
    gdf_voronoi['voronoi_geom'] = gdf_voronoi['geometry']
    parish_dataframe['parish_geom'] = parish_dataframe['geometry']
    
    # dissolve
    gdf_voronoi = pd.merge(gdf_voronoi,parish_dataframe[['Parish','parish_geom']],
                           how='left',on='Parish')
    
    # dropna/empty
    gdf_voronoi = gdf_voronoi[~gdf_voronoi.parish_geom.isna()].reset_index(drop=True)
    
#     for i,row in gdf_voronoi.iterrows():
#         try:
#             (row.voronoi_geom.buffer(0)).intersection(row.parish_geom.buffer(0))
#         except:
#             print('Problem at index: ' + str(i))
#             print('--> Voronoi geom: ' + str(row.voronoi_geom))
#             print('--> Parish geom: ' + str(row.parish_geom))
    
    try:
        gdf_voronoi['geometry'] = gdf_voronoi.progress_apply(lambda row: \
                    (row.voronoi_geom.buffer(0)).intersection(row.parish_geom.buffer(0)),axis=1)
    except:
       print('FAILED: ' + unique_parish)
    
    combined_voronoi.append(gdf_voronoi)
    combined_nodes.append(nodes_dataframe)
    
voronois = gpd.GeoDataFrame( pd.concat( combined_voronoi, ignore_index=True) )
voronois = voronois[['id','Parish','geometry']]
voronois = voronois[~voronois.geometry.is_empty].reset_index(drop=True)
voronois.to_file(driver='ESRI Shapefile',filename='../_tmp/voronoi-clipped.shp')

nodes = gpd.GeoDataFrame( pd.concat( combined_nodes, ignore_index=True) ) 
nodes.to_file(driver='ESRI Shapefile',filename='../_tmp/voronoi-nodes.shp')

print('Done!')

In [ ]:
voronois

In [ ]:
nodes.set_index('id')['population'].to_dict()

In [ ]:
population_dataframe = gpd.read_file('../data/population-russell/population.gpkg')

In [ ]:
population_dataframe